In [1]:
import os
from pyspark import SparkConf, SparkContext

# Report

本次作業要求用 Map-Reduce，完成 LSH 的計算，以下將依據三個步驟分別講解：

## Shingling
Convert documents to sets

### Mapper: readData
> **Input** `file_data(file_path, row_data)`
>
> **Output** `[(documentIdx1, [encodedStr1, encodedStr2, ....]), (documentIdx2, [encodedStr1, encodedStr2, ....]), ...]`

初始化 PySpark，讀入檔案後，首先將檔案逐行拆分。由於本次需分別讀入 100 個不同的 txt 檔，又必須記載下各自檔案的檔名 id ，我使用 `wholeTextFiles` 進行檔案讀取。在 `ReadData` Mapper 中，除了先提取 file index 外，便開始進行 k-Shingling 的操作。由於本次作業要求為 3 個 words Shingling ，我透過
```python
for first, second, third in zip(wordList, wordList[1:], wordList[2:])
```
，依序窮舉所有可能的 3 words combination，並透過 `binascii.crc32` ，將每 3 word combination encode 成 32 bit integer 長的 hash 值。每個 encode 出來的 integer，可對應到 Shingling Matrix 的 Row index。 最終，將 encode 的結果，與 file idx 合併輸出。

### Mapper: rowDocPairs
> **Input** `[(documentIdx1, [encodedStr1, encodedStr2, ....]), (documentIdx2, [encodedStr1, encodedStr2, ....]), ...]`
>
> **Output** `[(encodedStr1, [documentIdx1]), (encodedStr2, [documentIdx1]), (encodedStr3, [documentIdx1]), ...]`

為了方便後續操作，這步驟將資料進行統整，以 `encodedStr` 當作 key ，並將 documentIdx 包成 `list`，放在 value，方便後續 reducer 進行 value list combine。後續的
```python
.reduceByKey(lambda x, y: x+y).sortBy(lambda x: x[0], ascending=True)
```
則會根據 key `encodedStr`，找出其他也包含此字串組合的 documents。找齊後，重新根據 `encodedStr(int)` 重新以小到大排序。

In [2]:
import binascii
import re
import random


# Generate [(fileIdx1, [encodedStr1, encodedStr2, ....]), (fileIdx2, [encodedStr1, encodedStr2, ....]), ...]
def readData(data):
    filePath = data[0]
    fileIdx = int(re.search('athletics/(.+?).txt', filePath).group(1))
    lines = data[1]
    splittedItems = []
    for perLine in lines.splitlines():
        wordList = perLine.split(' ')
        for first, second, third in zip(wordList, wordList[1:], wordList[2:]):
            combinedStr = first + second + third;
            encodedStr = binascii.crc32(combinedStr.encode('utf8'))
            splittedItems.append(encodedStr)
#             splittedItems.append([first, second, third])
    return (fileIdx, splittedItems)

# Generate [(encodedStr1, [fileIdx1]), (encodedStr2, [fileIdx1]), (encodedStr3, [fileIdx1]), ...]
def rowDocPairs(data):
    expandedHashStr = []
    for perHashStr in data[1]: expandedHashStr.append((perHashStr, [ data[0] ]))
    return expandedHashStr


# 1. First Step: Shingling
conf = SparkConf().setMaster("local").setAppName("lsh")
sc = SparkContext.getOrCreate(conf=conf)
originalData = sc.wholeTextFiles("./athletics/*.txt").map(readData)

# Last Reducer Generate [(encodedStr1, [fileIdx1, fileIdx2, ...]), (encodedStr2, [fileIdx1, fileIdx2, ...]), ...]
splittedData = originalData.flatMap(rowDocPairs).reduceByKey(lambda x, y: x+y).sortBy(lambda x: x[0], ascending=True)

## Minhashing
Covert large sets to short signatures, while preserving similarity.

### Mapper: readjustHashKey
> **Input** `[(encodedStr1, [fileIdx1, filIdx2]), (encodedStr2, [fileIdx1]), (encodedStr3, [fileIdx1, filIdx3]), ...]`
>
> **Output** `[(1(rowKey), [fileIdx1, fileIdx2]), (2, [fileIdx1]), (3, [fileIdx1, fileIdx3]), ...]`

由於上面整理出的 key `encodedStr`，是不連續的 `integer` index。此 Mapper 主要目標在於，讓 key 重新由 `1` 開始排序。

### Mapper: minHashing
> **Input** `[(1, [fileIdx1, fileIdx2]), (2, [fileIdx1]), (3, [fileIdx1, fileIdx3]), ...]`
>
> **Output** `[((rowKey, documentIdx, hashFuncIdx), hashFuncValue)), ... ]`

在此 mapper 中，我仿照講義 `p.44` 的做法進行實作。首先會先生成 100 個 `hashFunction`(根據講義，我再 mod 一個質數 `31583`)，
```python
for coefficient in range(1, 101):
    hashFuncs.append(((coefficient*2)*data[0] + 1)%31583)
```
生成的 hash 依序為：
```
(2*rowIdx+1)%31583
(4*rowIdx+1)%31583
(6*rowIdx+1)%31583
...
(200*rowIdx+1)%31583
```
最終再 mod N (shingle 數量) `totalCount`

接著，再進入
```python
for documentIdx in data[1]:
    for hashFuncIdx, hashFunc in enumerate(hashFuncs):
        result.append(((data[0], documentIdx, hashFuncIdx), hashFunc))
```
簡單來說，因為我的實作中，value 存放的是 **任一 encodedStr** 的 **所有 Documents** ，因此 ``data[1]`` 掃到的所有 **DocumentID**，都代表 Shingling Matrix 中，該 encodedStr Row 與 Document Column 的值為 `1` 。而當每找到對應的 hashFunc Value 後，就進行儲存。

然則根據講義，我們僅需保留 **最小** 的 hashFunc Value 值。也因此，我執行了
```python
.reduceByKey(lambda x, y: x if x < y else y)
```
使**任一 HashFunc，對應到任一 Document 與 EncodedStr Row，只會紀錄下最小的值**。此步驟結束後，即會產生 `signature matrix`。

In [3]:
# Readjust key starting with 1, generate [(1, [fileIdx1, fileIdx2]), (2, [fileIdx1]), (3, [fileIdx1, fileIdx3]), ...]
reAdjustIndex = 0
totalCount = 0
def readjustHashKey(data):
    global reAdjustIndex
    reAdjustIndex = reAdjustIndex + 1
    return (reAdjustIndex, data[1])

def minHashing(data):
    global totalCount
    hashFuncs = []
    result = []
    for coefficient in range(1, 101):
        hashFuncs.append((((2*coefficient)*data[0] + 1)%31583)%totalCount)
    for documentIdx in data[1]:
        # If documentIdx is existed, it means that c has 1 in this row
        for hashFuncIdx, hashFunc in enumerate(hashFuncs):
            result.append(((data[0], documentIdx, hashFuncIdx), hashFunc))
    return result

# 2. Second Step: Min-Hashing
newSplittedData = splittedData.map(readjustHashKey)
totalCount = newSplittedData.keys().count()
result = newSplittedData.flatMap(minHashing).reduceByKey(lambda x, y: x if x < y else y)

## Locality-sensitive hashing (LSH)
Focus on pairs of signatures likely to be from similar documents.

### Mapper: retidyForLsh
> **Input** `[((rowKey, documentIdx, hashFuncIdx), hashFuncValue)), ... ]`
>
> **Output** `[((newHashKey, documentIdx), value)), ... ]`

根據題幹，我們需設定兩個 `band` 為一個 `row`。也因此，我重新計算 `hashFuncIdx` 如下：
```python
newHashFuncIdx = math.floor(hashFuncIdx/2)
```
也因此 `0,1` 會是新的 row `0` ，`2, 3` 會是新的 row `1` ，以此類推。

而對於每個 band ，每個 document column 必須重新進行 rehash，我先透過
```python
.reduceByKey(lambda x, y: x + y)
```
將上下兩個的 signature 相加，方便後續做 hashing 。


In [4]:
import math

def retidyForLsh(data):
    documentIdx = data[0][1]
    hashFuncIdx = data[0][2]
    value = data[1]
    newHashFuncIdx = math.floor(hashFuncIdx/2)
    return ((newHashFuncIdx, documentIdx), value)

# 兩個 row 的資料是要相加起來再去 hash(目前是這樣)
tidiedData = result.map(retidyForLsh).reduceByKey(lambda x, y: x + y)

### Mapper: hashLsh 
> **Input** `[((newHashKey, documentIdx), value)), ... ]`
>
> **Output** `[(bucketIdx1, [documentIdx1, documentIdx2, ... ]), ... ]`

接續前步驟，我們將剛相加的 signature 值，重新 hash，並放入對應的 hash bucket。根據講義，bucket 數量為越多越好，我將 `K (Number of Bucket)` 設為 `10000` ，最終 hash 的方法則為 `(2*originalValue + 5)%K`。

經過 `hashLsh` 後的值，為 `[ (HashBucket1, [documentidx1]), (HashBucket1, [documentidx2]), ...]`，屬於同一個 `HashBucket` 的 Documents 還需合併記載，因此透過
```python
.reduceByKey(lambda x, y: x + y)
```
將 value 的 list 進行合併。

### Mapper: removeDuplicatedDocumentIdx
> **Input/Output** `[ (HashBucket1, [documentidx1, documentidx2, documentidx3, ... ]), (HashBucket2, [documentidx1, documentidx2, documentidx3, ... ]), ...]`

因為同一份文件中，相同的字詞組合可能重複出現。同時，不同的 row 在計算 hashing、放入 bucket 時，會將相同的 Document 重複紀錄。導致目前的資料`values(documentIDs)`仍會重複。我使用 `removeDuplicatedDocumentIdx mapper` ，讓重複的 `DocumentIdx` 移除。

而當任一 Bucket，**內部存放的 Document 數量，沒有超過 1 個時**，我透過 `.filter(lambda x: len(x[1]) > 1)` 將該 bucket 移除。




In [5]:
def hashLsh(data):
    documentIdx = data[0][1]
    hashResult = (2*data[1] + 5)%10000
    return (hashResult, [documentIdx])

def removeDuplicatedDocumentIdx(data):
    # Since some document has repeatedly appear in one single hashResult, we need to remove that.
    return (data[0], list(set(data[1])))

# LSH Process
# Output: [ (HashBucket1, [documentidx1, documentidx2, documentidx3, ... ]), (HashBucket2, [documentidx1, documentidx2, documentidx3, ... ]), ...]
finalResult = tidiedData.map(hashLsh).reduceByKey(lambda x, y: x + y).map(removeDuplicatedDocumentIdx).filter(lambda x: len(x[1]) > 1)

### Mapper: getPossibleDocumentsPairs 
> **Input** `[ (HashBucket1, [documentidx1, documentidx2, documentidx3, ... ]), (HashBucket2, [documentidx1, documentidx2, documentidx3, ... ]), ...]`
>
> **Output** `[ (similarDocument1, similarDocument2), (similarDocument1, similarDocument3), ... ]`

找出所有**超過 1 個 Document**的 bucket 後，便可將該 bucket 的 document 進行兩兩配對，窮舉出所有的組合。在此 Mapper 中，我們強制小的 DocumentId 放前面，大的 DocumentId 放後面，方便後續比對是否有重複。

接著，再透過
```
.flatMap(lambda data: [ (data[0], data), (data[1], data) ])
```
將資料展開成  
```
[ (similarDocument1, (similarDocument1, similarDocument2)), (similarDocument2, (similarDocument1, similarDocument2)), ... ]
```
方便後續和 Document rowData 進行 `join`。

### Join + calculateSimilarity(Reducer)
> **Input** `[ (similarDocument1, (similarDocument1, similarDocument2)), (similarDocument2, (similarDocument1, similarDocument2)), ... ]`
>
> **Output** `[ ((similarDocument1, similarDocument2), jaccard), ((similarDocument1, similarDocument3), jaccard), ... ]`

由於已找出所有的 Similar Document Pairs ，進入 LSH 最後一步驟，將相似的 Document 統合計算 Jaccard Similarity。首先，因為目前 key 已經是 `documentIdx` ，我們和 originalData 進行 `join`，讓 key / val 為
```
(similarDocument1, ((similarDocument1, similarDocument2), (encodedStr1, encodedStr2, encodedStr3, ...))
(similarDocument2, ((similarDocument1, similarDocument2), (encodedStr1, encodedStr2, encodedStr3, ...))
...
```
從上方可見，當取出 `values()` 時，便可拿到
```
((similarDocument1, similarDocument2), (encodedStr1, encodedStr2, encodedStr3, ...)) --> document 1
((similarDocument1, similarDocument2), (encodedStr1, encodedStr2, encodedStr3, ...)) --> document 2
```
再透過 reducer `calculateSimilarity` ，將兩個 document 的 encodedStr 進行 Jaccard Similarity 計算。計算方法為 `intersection/union`，並輸出成最終值。這邊計算的是 `col/col similarity`。

最後重新依據相似度進行排列，即完成本次作業。


In [6]:
def getPossibleDocumentsPairs(data):
    result = []
    for x in data[1]:
        for y in data[1]:
            if (x != y):
                if (x < y): result.append((x, y))
                else: result.append((y, x))
    return list(set(result))

def calculateSimilarity(x, y):
    unionResult = set(x).union(set(y))
    intersectionResult = set(x).intersection(set(y))
    return len(intersectionResult) / len(unionResult)

# Output: [ (similarDocument1, similarDocument2), (similarDocument1, similarDocument3), ... ]
finalResult = finalResult.flatMap(getPossibleDocumentsPairs).distinct()
# Output: [ (similarDocument1, (similarDocument1, similarDocument2)), (similarDocument2, (similarDocument1, similarDocument2)), ... ]
finalResult = finalResult.flatMap(lambda data: [ (data[0], data), (data[1], data) ])

finalResult = finalResult.join(originalData).values().reduceByKey(calculateSimilarity).sortBy(lambda x: x[1], ascending=False)

for item in finalResult.collect()[0:10]:
    print('%s: %f %s\n'%(item[0], item[1]*100, '%'))
# sc.stop()

(52, 84): 100.000000 %

(12, 20): 100.000000 %

(30, 35): 70.958084 %

(48, 49): 48.356808 %

(14, 40): 39.898563 %

(23, 48): 11.625709 %

(81, 82): 10.000000 %

(79, 82): 8.844765 %

(79, 81): 8.284024 %

(19, 21): 4.657534 %

